In [425]:
import pandas as pd
import numpy as np
import pretty_midi
import music21
import re

In [433]:
import os
os.chdir("/Users/desireewaugh/Desktop/MIT/Courses/6.883 - Modeling with ML/Projects/Final Project/MusicGenerator/data")

## Play abc file

In [50]:
# Run "pip install music21" in your chosen directory
from music21 import *

In [438]:
music_file = converter.parse('rockethornpipe-1.abc')

In [5]:
# Need to download pygame to play files: "pip install pygame"
sp = midi.realtime.StreamPlayer(music_file)
sp.play()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## With midi files

In [4]:
# Data from: https://magenta.tensorflow.org/datasets/groove#dataset
# Possible dataset: https://drive.google.com/file/d/14e0MCJD7RH_m7CpsFZWPIpO0WgQrwi64/view

In [418]:
os.chdir("./groove/drummer1/session1")
drum_file = pretty_midi.PrettyMIDI("4_jazz-funk_116_beat_4-4.mid")

In [208]:
midi = pretty_midi.PrettyMIDI("hotel_california.mid")
piano_midi = midi.instruments[2]
piano_roll = piano_midi.get_piano_roll()

piano_roll

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Featurization Option 1: Matrix with note start, end, pitch, and velocity
### Each row is a separate observation
### pretty_midi library

In [430]:
# Reference: https://www.audiolabs-erlangen.de/resources/MIR/FMP/C1/C1S2_MIDI.html
def encode_midi(midi_file):
    midi_info = []
    for instrument in midi_file.instruments:
        for note in instrument.notes:
            start = note.start
            end = note.end
            pitch = note.pitch
            velocity = note.velocity
            midi_info.append([start, end, pitch, velocity, 
                              instrument.program, instrument.is_drum, instrument.name])

    df = pd.DataFrame(midi_info, columns=['Start', 'End', 'Pitch', 'Velocity', 
                                                   'Program', 'is_drum', 'Name'])
    midi_matrix = df[['Start', 'End', 'Pitch', 'Velocity', 'Program', 'is_drum']].values
    return midi_matrix
    
test_matrix = encode_midi(midi)
test_matrix

array([[1.7814931250000001, 1.7896100000000001, 36, 103, 0, True],
       [1.7814931250000001, 1.7896100000000001, 42, 72, 0, True],
       [1.7814931250000001, 1.7896100000000001, 57, 84, 0, True],
       ...,
       [26.72465, 26.919455, 74, 92, 18, False],
       [26.72465, 26.822052499999998, 47, 97, 28, False],
       [26.749000625, 26.919455, 59, 110, 25, False]], dtype=object)

In [431]:
# Create midi file from a dataframe
# Reference: http://craffel.github.io/pretty-midi/

def decode_midi(midi_matrix, instrument_name):
    created_midi = pretty_midi.PrettyMIDI()

    # Add instruments
    for i in range(midi_matrix.shape[0]):
        instrument = pretty_midi.Instrument(program=int(midi_matrix[i,4]), 
                                            is_drum=bool(midi_matrix[i,5]),
                                            name=instrument_name)

        note = pretty_midi.Note(  
            start=midi_matrix[i, 0], 
            end=midi_matrix[i, 1],
            pitch=int(midi_matrix[i, 2]),
            velocity=int(midi_matrix[i, 3]))
        instrument.notes.append(note)
        created_midi.instruments.append(instrument)

    return created_midi

new_midi = decode_midi(test_matrix, "Acoustic Grand Piano")

# Save as midi file
new_midi.write("test_midi.midi")

In [437]:
# Play written file - this does not sounds great, but it might be good enough to train a NN
b = converter.parse("test_midi.midi")
sp = music21.midi.realtime.StreamPlayer(b)
sp.play()

In [436]:
# Play actual file - sounds much better
b = converter.parse("hotel_california.mid")
sp = music21.midi.realtime.StreamPlayer(b)
sp.play()

KeyboardInterrupt: 

In [432]:
# Classical files
os.chdir("/Users/desireewaugh/Desktop/MIT/Courses/6.883 - Modeling with ML/Projects/Final Project/MusicGenerator/data/classical")

beethoven = converter.parse("beethoven_hammerklavier_1.mid")
sp = music21.midi.realtime.StreamPlayer(beethoven)
sp.play()

## Train Neural Network

In [435]:
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dense, Dropout, Activation, Flatten

Using TensorFlow backend.


In [ ]:
# Reference: https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

model = Sequential()
model.add(LSTM())


model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

## Featurization Option 2: Encoding with Offsets
### music21 library

In [325]:
# Source: https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

def encode_midi2(file_path):
    midi_file = music21.converter.parse(file_path)
    parts = music21.instrument.partitionByInstrument(midi_file)

    pitches = []
    volumes = []
    offsets = []

    # Get list of each note in the midi file
    for part in parts:
        for element in part:
            if type(element) == music21.note.Note:
                pitches.append(str(element.pitch))
                volumes.append(element.volume.velocity)
                offsets.append(element.offset)
            if type(element) == music21.chord.Chord:
                for note in element:
                    note.offset = element.offset
                    pitches.append(str(note.pitch))
                    volumes.append(note.volume.velocity)
                    offsets.append(note.offset)

    df = pd.DataFrame(list(zip(pitches, volumes, offsets)), 
                      columns=['Pitch', 'Volume', 'Offset'])

    
    music_df = pd.pivot_table(df, values='Volume', index='Pitch', columns='Offset')
    music_df = music_df.fillna(0)

    # Evenly space columns
    col_names = np.arange(music_df.columns[0], music_df.columns[-1]+0.25, 0.25)

    # Impute columns that are missing based on previous column
    for col in col_names: 
        if col not in music_df.columns:
            music_df[col] = music_df[col - 0.25]

    # Delete columns that don't follow spacing interval 
    for col in music_df.columns:
        if col not in col_names:
            del music_df[col]

    return music_df


my_df = encode_midi2("hotel_california.mid")
my_df.iloc[1,0]



84.0

In [356]:
# THIS DOES NOT WORK SO FAR
def decode_midi2(dataframe):
    midi_file = music21.stream.Score()
    
    # We will create a midi file with just one part
    part = music21.stream.Part()
    
    rows, cols = dataframe.shape
    
    # Add the notes from the dataframe to the part object
    for r in range(0, rows):
        for c in range(0, cols):
            if dataframe.iloc[r,c] != 0:
                note = music21.note.Note(nameWithOctave=dataframe.index[r])
                note.pitch = music21.pitch.Pitch(dataframe.index[r]) # pitch = dataframe.indec[r]
                note.volume = music21.volume.Volume(velocity=dataframe.iloc[r,c]) # velocity = dataframe.iloc[r,c]
                note.offset = dataframe.iloc[r,c]

                part.append(note)
            
    midi_file.insert(part)
    
    return midi_file
    
            
my_file = decode_midi2(my_df)


In [287]:
b = converter.parse("hotel_california.mid")
parts = music21.instrument.partitionByInstrument(b)

for part in parts:
    for element in part:
        print(element)

Electric Bass

<music21.tempo.MetronomeMark Quarter=77.0>
<music21.chord.Chord C2 F#2 A3>
<music21.chord.Chord G2 D4 G4>
<music21.note.Note D>
<music21.note.Rest rest>
<music21.chord.Chord D3 G5 B5>
<music21.note.Note D>
<music21.chord.Chord D2 D4 G4>
<music21.chord.Chord G#2 G4 B4>
<music21.chord.Chord G1 D5 B5 D6>
<music21.note.Rest rest>
<music21.note.Note G>
<music21.note.Rest rest>
<music21.chord.Chord G3 B3>
<music21.note.Rest rest>
<music21.chord.Chord F#2 G4 B4>
<music21.chord.Chord G4 D5 B5 D6>
<music21.chord.Chord D4 G4 G3>
<music21.note.Rest rest>
<music21.chord.Chord E2 F#2>
<music21.chord.Chord G4 D5 B5 D6>
<music21.chord.Chord D4 G4 B3 D4>
<music21.chord.Chord G#2 G4 B4>
<music21.chord.Chord F#2 G4 B4>
<music21.chord.Chord D2 D5 B5 D6 G3 B3>
<music21.note.Note G>
<music21.note.Note G>
<music21.chord.Chord C2 F#2>
<music21.chord.Chord G2 G4 D5 B5 D6>
<music21.chord.Chord D4 G4>
<music21.chord.Chord B3 D4>
<music21.note.Rest rest>
<music21.chord.Chord G#2 G4 B4>
<music21.ch

In [371]:
# Source: https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

def encode_midi3(file_path):
    midi_file = music21.converter.parse(file_path)
    parts = music21.instrument.partitionByInstrument(midi_file)

    notes = []
    volumes = []

    # Get list of each note in the midi file
    for part in parts:
        for element in part:
            if type(element) == music21.note.Note:
                notes.append(str(element.pitch))
                volumes.append(element.volume.velocity)
            if type(element) == music21.chord.Chord:
                notes.append('.'.join(str(n) for n in element.normalOrder))
                volumes.append(element.volume.velocity)
                
    note_dict = dict((note, num) for num, note in enumerate(notes))

    note_ints = [note_dict[note] for note in notes]
    return note_ints, volumes


my_result = encode_midi3("hotel_california.mid")
my_result


([276,
  173,
  2,
  178,
  107,
  173,
  233,
  178,
  246,
  243,
  241,
  178,
  173,
  255,
  178,
  178,
  233,
  241,
  178,
  246,
  228,
  244,
  178,
  173,
  165,
  233,
  178,
  241,
  178,
  236,
  171,
  172,
  173,
  233,
  175,
  246,
  241,
  243,
  178,
  244,
  207,
  41,
  42,
  198,
  202,
  198,
  212,
  212,
  212,
  207,
  255,
  243,
  198,
  202,
  212,
  212,
  212,
  260,
  243,
  244,
  60,
  207,
  201,
  202,
  243,
  212,
  212,
  67,
  207,
  212,
  70,
  253,
  212,
  212,
  244,
  272,
  253,
  264,
  272,
  257,
  275,
  220,
  275,
  220,
  272,
  255,
  269,
  257,
  275,
  269,
  260,
  244,
  272,
  264,
  257,
  275,
  269,
  97,
  272,
  99,
  220,
  269,
  220,
  275,
  244,
  277,
  220,
  107,
  108,
  207,
  207,
  120,
  115,
  140,
  140,
  115,
  207,
  255,
  118,
  278,
  120,
  140,
  140,
  260,
  124,
  125,
  207,
  165,
  270,
  270,
  131,
  131,
  140,
  133,
  134,
  207,
  171,
  139,
  138,
  139,
  140,
  276,
  178,
  246,
 